<a href="https://colab.research.google.com/github/aletorales/SeminarioCienciaDeDatos/blob/master/WebScrapping_Banxico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import re
import os
import pandas as pd
import matplotlib.pyplot as plt
from requests_html import HTMLSession
from collections import defaultdict
from unidecode import unidecode
from datetime import datetime

In [0]:
%config InlineBackend.figure_format = "retina"
plt.rcParams["figure.figsize"] = (12, 4)

In [0]:
url = ("https://www.banxico.org.mx/publicaciones-y-prensa/"
      "anuncios-de-las-decisiones-de-politica-monetaria/"
      "anuncios-politica-monetaria-t.html")

In [0]:
with HTMLSession() as sess:
    r=sess.get(url)
r

In [0]:
tbl= r.html.find("table", first=True)
tbl
#table table-striped bmtableview

In [0]:
len(tbl.find("tr"))

In [0]:
minutas=tbl.find("tr")

In [0]:
#extrae fecha y abstract de cada minuta
qdate="td[class='bmdateview']"
qabst="td[class='bmtextview']"

minutas[0].find(qdate,first=True).text
minutas[0].find(qabst,first=True).text

In [0]:
#extrae la url 
qurl="a[onclick]"
minutas[0].find(qurl,first=True).absolute_links
purl,*_=minutas[0].find(qurl,first=True).absolute_links
purl

In [0]:
#Extrayendo todo 
qdate="td[class='bmdateview']"
qabst="td[class='bmtextview']"
qurl="a[onclick]"

data=[]
for minu in minutas:
    date= (minu.find(qdate, first=True)
                .text)
    abst= (minu.find(qabst, first=True)
               .text)
    purl, *_= (minu.find(qurl, first=True)
                 .absolute_links)
    data_fr={
        "date":datetime.strptime(date,"%d/%m/%y"),
        "text": abst,
        "url": purl
    }
    data.append(data_fr)

In [0]:
comunicados=pd.DataFrame(data)
comunicados

In [0]:
###########Prueba
fecha=comunicados["date"][0]
fecha
datetime.strptime(fecha,"%d/%m/%y")

In [0]:
#Filtra después del 15-02-2008 y acomoda de manera ascendente
comunicados_f=(comunicados[comunicados["date"]>=datetime(2008, 2, 15)].set_index("date")
                                                                      .sort_index())

In [0]:
#Convierte archivo a pickle y lo lee
comunicados_f.to_pickle("comunicados-banxico.pkl")
info_df=pd.read_pickle("comunicados-banxico.pkl")
info_df.head()

In [0]:
#Extrae del texto si la tasa de mantiene incrementa o disminuye
extract=pd.Series(info_df["text"])
estado_tasa=extract.str.extract(pat='(mantiene|incrementa|disminuye)')
tasa=pd.DataFrame(estado_tasa)

In [0]:
#full_df=info_df.join(tasa, how="left")
full_df=info_df.assign(category=tasa)
full_df

In [0]:
#Cuenta cuántas veces incrementa, disminuye o se mantiene la tasa 
(full_df.groupby("category").count()
                            .drop("url",axis=1))

In [0]:
(full_df.groupby("category").count()
                            .drop("url",axis=1)).plot.bar(legend=False,figsize=(12, 4))

In [0]:
def convert(df):
    for i in range(len(df)):
        if df[0][i]=="mantiene":
            df[0][i]==0
        elif df[0][i]=="disminuye":
            df[0][i]==-1
        else:
            df[0][i]==1
    return df

In [0]:
tabla_2=convert(estado_tasa)

In [0]:
.assign(year=lambda v: v["paymentDate"].astype("datetime64[Y]")